In [7]:
from bs4 import BeautifulSoup
import  requests, json,pickle,tqdm

def get_superclass_and_glycoside(smiles):
    # smiles = 'CC1C(O)CC2C1C(OC1OC(COC(C)=O)C(O)C(O)C1O)OC=C2C(O)=O'
    url = f"https://npclassifier.ucsd.edu/classify?smiles={smiles}"
    # url = f"https://npclassifier.gnps2.org/classify?smiles={smiles}"
    response = requests.get(url)
    json_dat = json.loads(response.content)
    superclass_results = json_dat['class_results']
    isglycoside = json_dat['isglycoside']
    return superclass_results, isglycoside

In [9]:
for split in ['val', 'test', "train"]:
    count = 0
    superclass_results = {}
    smiles_pkl = pickle.load(open(f'/workspace/SMILES_dataset/{split}/SMILES/index.pkl','rb'))
    for k,v in (smiles_pkl.items()):
        count+=1
        print(v)
        if count == 100: break
        print( get_superclass_and_glycoside(v) )
    break    
    # pickle.dump(superclass_results,open(f'/workspace/SMILES_dataset/{split}/Superclass/index.pkl','wb'))
        
        

Br.CN1CCc2cc(O)cc3c2C1Cc1ccc(O)c(O)c1-3
(['Aporphine alkaloids', 'Isoquinoline alkaloids'], False)
Brc1cc(Br)c2cc[nH]c2c1
(['Simple indole alkaloids'], False)
Brc1ccc2[nH]c3cnccc3c2c1
(['Carboline alkaloids'], False)
Brc1ccc2[nH]ccc2c1
(['Simple indole alkaloids'], False)
C#CC#CC#CC=CCO
([], False)
C#CC#CC1OC(=O)Cc2ccccc21
([], False)
C#CC(=O)C=CCCCCCCCCCC=CCCCCCCCCCCCC(O)C#CC#CCO
([], False)
C#CC(=O)C=CCCCCCCCCCCC=CC(O)C#C
([], False)
C#CC(Br)C1CC(O)C(CC2OC(CC)C(Br)CC2Br)O1
([], False)
C#CC(Br)C1CC2OC3(CCC(C(Br)CC)O3)C(Br)C2O1
([], False)
C#CC(C)c1cc2ccccc2c(=O)o1
([], False)
C#CC(O)C=CC(O)CCCC(O)C(O)C#CC(O)C#CCCCCC(=O)CCCCCC=CC(O)CCCCCCCCCCCCC(O)C(O)C(O)C#CC(=O)O
([], False)
C#CC(O)C=CC(O)CCCC(O)C(O)C#CC(O)C#CCCCCC(=O)CCCCCC=CC(O)CCCCCCCCCCCCCC(O)C(O)C#CC(=O)O
([], False)
C#CC(O)C=CC(O)CCCCCCCCC=CCCCCCCCCCCCC(O)C=CC(O)C#C
([], False)
C#CC(O)C=CC(O)CCCCCCCCC=CCCCCCCCCCCCCC(O)C=CC(O)C#C
([], False)
C#CC(O)C=CCCCCC=CC=CC#CCCCC(O)C=C
([], False)
C#CC(O)C=CCCCCCCC#CC(O)C=CC(O)C#CCCCCCCC=C

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [6]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

def fetch_data(idx_and_url):
    index, url = idx_and_url
    try:
        response = requests.get(url)
        print(f'url {url}\n response: {response.content}')
        json_dat = json.loads(response.content)
        
        superclass_results = json_dat['class_results']
        isglycoside = json_dat['isglycoside']
        
        # if len(superclass_results) == 0:
        #     print(f"Empty results for {url}")
        
        return index, superclass_results, isglycoside
    except Exception as e:
        return index, f"Error fetching {url}: {e}"
        


In [10]:
for split in ['val', 'test', "train"]:
    superclass_results = {}
    smiles_pkl = pickle.load(open(f'/workspace/SMILES_dataset/{split}/SMILES/index.pkl','rb'))
    
    indices_n_urls = [(k, f"https://npclassifier.gnps2.org/classify?smiles={v}") for k,v in smiles_pkl.items()][:50]
    results = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_data, idx_and_url) for idx_and_url in indices_n_urls]
        for future in as_completed(futures):
            results.append(future.result())
        
    # pickle.dump(superclass_results,open(f'/workspace/SMILES_dataset/{split}/Superclass/index.pkl','wb'))
    break    
        

url https://npclassifier.gnps2.org/classify?smiles=Brc1ccc2[nH]c3cnccc3c2c1
 response: b'{"class_results": ["Carboline alkaloids"], "superclass_results": ["Tryptophan alkaloids"], "pathway_results": ["Alkaloids"], "isglycoside": false}'
url https://npclassifier.gnps2.org/classify?smiles=Brc1ccc2[nH]ccc2c1
 response: b'{"class_results": ["Simple indole alkaloids"], "superclass_results": ["Tryptophan alkaloids"], "pathway_results": ["Alkaloids"], "isglycoside": false}'
url https://npclassifier.gnps2.org/classify?smiles=C#CC(=O)C=CCCCCCCCCCC=CCCCCCCCCCCCC(O)C#CC#CCO
 response: b'{"class_results": [], "superclass_results": [], "pathway_results": [], "isglycoside": false}'
url https://npclassifier.gnps2.org/classify?smiles=C#CC#CC1OC(=O)Cc2ccccc21
 response: b'{"class_results": [], "superclass_results": [], "pathway_results": [], "isglycoside": false}'
url https://npclassifier.gnps2.org/classify?smiles=C#CC#CC#CC=CCO
 response: b'{"class_results": [], "superclass_results": [], "pathway_resu

In [14]:
some_smiles = list(smiles_pkl.values())
len(some_smiles), len(set(some_smiles))

(13756, 11151)

In [17]:
import os
files = os.listdir('/workspace/SMILES_dataset/test/HYUN_FP/')
len(files)

13630

In [16]:
results

[(2, 'Carboline alkaloids', False),
 (3, 'Simple indole alkaloids', False),
 (0, 'Aporphine alkaloids', False),
 (1, 'Simple indole alkaloids', False),
 (4,
  'Error fetching https://npclassifier.gnps2.org/classify?smiles=C#CC#CC#CC=CCO: list index out of range')]

In [7]:

import json
url = 'https://npclassifier.ucsd.edu/classify?smiles=C#CC#CC#CC=CCO'
try:
    response = requests.get(url)
    json_dat = json.loads(response.content)
 
    print(  json_dat)
except Exception as e:
    print( f"Error fetching {url}: {e}")

{'class_results': [], 'superclass_results': [], 'pathway_results': [], 'isglycoside': False, 'fp1': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
# index, url = indices_n_urls[0]
url = 'https://npclassifier.gnps2.org/classify?smiles=C#CC#CC#CC=CCO'
response = requests.get(url)
response.content

b'{"class_results": [], "superclass_results": [], "pathway_results": [], "isglycoside": false}'